In [1]:
%load_ext autoreload
%autoreload 2

In [120]:
import pandas as pd 
import numpy as np
from preprocessing.cleaning_utils import *
from functools import partial
import torch 

from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict
import transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import logging
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import spacy
import spacy
from spacy import displacy
from spacy.symbols import nsubj, VERB, nmod, ccomp, punct, conj
from spacy.tokens import Doc, Span, Token
logging.set_verbosity_warning()

In [3]:
# !python -m torch.utils.collect_env

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
mimic_dir = "/home/vs428/project/MIMIC/files/mimiciii/1.4/"
n2c2_dir = "/home/vs428/project/n2c2/2022/N2C2-AP-Reasoning/"
n2c2_data_dir =  "/home/vs428/project/n2c2/2022/Data/"


In [6]:
classes = ['Not Relevant', 'Neither', 'Indirect', 'Direct']
features = Features({
    'ROW ID':Value("int64"),
    'HADM ID':Value("int64"),
    'Assessment':Value("string"),
    'Plan Subsection':Value("string"),
    "Relation":Value("string")
}) 

dataset = load_dataset("csv", data_files={
                            "train":n2c2_data_dir + "train.csv",
                            "valid":n2c2_data_dir + "dev.csv",
                        },

                       features=features)
# dataset = dataset.class_encode_column("Relation")
dataset = dataset.rename_column("Relation", "label")

Using custom data configuration default-b1948d86214b7517
Reusing dataset csv (/home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset['train'][23]

{'ROW ID': 711150,
 'HADM ID': 183333,
 'Assessment': '75 yo M with h/o cardiomyopahy (EF 15%), atrial fibrillation, pulmonary\n   embolism (on coumadin), adrenal insufficiency, and s/p several recent\n   admissions following episode of Klebsilella pneumonia complicated by\n   respiratory failure and septic shock. Presents to hospital with\n   syncope, hypoxia, hypotension and altered mental status.',
 'Plan Subsection': '# Adrenal insufficiency:\n   Patient on 10 mg hydrocortisone at home.\n   - Increase to 50 mg hydrocortisone given stress of acute illness (day 2\n   today)',
 'label': 'Indirect'}

In [10]:
from transformers import GPT2Tokenizer, GPTNeoForSequenceClassification


In [107]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


1

In [111]:
model = GPTNeoForSequenceClassification.from_pretrained("EleutherAI/gpt-neo-1.3B", num_labels=4, 
                                                        problem_type="single_label_classification",
                                                       pad_token_id=tokenizer.convert_tokens_to_ids("[PAD]"))



Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['transformer.h.17.attn.attention.bias', 'transformer.h.21.attn.attention.bias', 'transformer.h.1.attn.attention.bias', 'transformer.h.19.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'transformer.h.3.attn.attention.bias', 'score.weight', 'transformer.h.23.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.13.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'transformer.h.7.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [112]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 2048)

In [113]:
out = tokenizer(dataset['train'][43:45]['Assessment'], dataset['train'][43:45]['Plan Subsection'],
                                      truncation="longest_first",
                                    # padding=True,
                                      max_length=512,
                                      verbose=True)

In [125]:
generator = pipeline(task="text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

In [128]:
generator([dataset['train'][43]['Assessment'] +  dataset['train'][43]['Plan Subsection'], 
           dataset['train'][44]['Assessment'] +  dataset['train'][44]['Plan Subsection']])[0]

[{'label': 'LABEL_0', 'score': 0.45915326476097107},
 {'label': 'LABEL_1', 'score': 0.03891521692276001},
 {'label': 'LABEL_2', 'score': 0.478484570980072},
 {'label': 'LABEL_3', 'score': 0.02344704233109951}]

In [134]:
x = np.zeros(shape=(5,5))

In [138]:
x[4, :] = [1,2,3,4,5 ]

In [139]:
x

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 2., 3., 4., 5.]])

In [136]:
data_collator = DataCollatorWithPadding(tokenizer,
                                        max_length=512, 
                                        padding="max_length",
                                        return_tensors="pt")    


In [116]:
test = data_collator(out)

In [117]:
# tokenizer.decode(out['input_ids'])
model_out = model(**test)

In [118]:
model_out.logits

tensor([[ 1.5115, -0.9565,  1.5527, -1.4632],
        [ 0.1700, -2.3643,  3.2297, -0.9741]], grad_fn=<IndexBackward0>)

In [ ]:
dataset = dataset.map(partial(tokenize_function, tokenizer=tokenizer), batched=True)

# tokenizer(data['train'][23])

In [8]:
# # split dataset: 80% train, 10% validation, 10% test 
# train_testvalid = dataset['train'].train_test_split(test_size=0.2)
# # test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# split_dataset = DatasetDict({
#     'train': train_testvalid['train'],
#     'valid': test_valid['train'],
#     # 'test': test_valid['test'],
# })



In [9]:
# prompts = []
# for data in dataset['train']:
#     prompt_str = 'Assessment: "' + data['Assessment'] + '"\nPlan: "' + data['Plan Subsection'] + '"\nLabel: ' \
#         + str(data['label'])
#     prompts.append(prompt_str)
prompts = []
for data in dataset['train']:
    prompt_str = 'Assessment: ' + data['Assessment'] + '\nPlan: ' + data['Plan Subsection'] + '\nLabel: ' \
        + str(data['label'])
    prompts.append(prompt_str)
        

In [10]:
prompts[10]

'Assessment: Assesment: This is a 76 year-old female with a history of diabetes who\n   presents with pneumoperitoneum of unclear aetiology, complicated by\n   coagulopathy and rhabdomyolysis.\nPlan: # Acute renal failure - [**Month (only) 60**] be pre-renal or secondary to\n   rhabdomyolysis. Creatinine down from 1.7 to 1.3.\n   - Continue aggressive IV fluid hydration\n   - Monitor potassium (3.8 this AM)\n   - Renally dose antibiotics\nLabel: Indirect'

In [11]:
for idx, x in enumerate(prompts):
    print(x)
    print("\n")
    if idx % 5 == 0 and idx > 5:
        break

Assessment: 51 yr old F with a history of 3V CAD, confirmed on C. cath during this
   admission, EF of 40%, who is transferred to CCU for monitoring of
   recurrent chest pain. Plan for CABG today to revascularize due to
   3-vessel disease.
Plan: # CORONARIES: Patient with 3 vessel disease on cath. Previously found
   to be the case in [**2174**], but had been treated medically. On the floor,
   patient with recurrent chest pain and EKG changes, concerning for
   further ischemia. Chest pain has been controlled on nitro gtt. Patient
   in CCU for monitoring; plan for CABG today.
   - continue nitro gtt
   - if has further CP, would uptitrate nitro, and contact [**Name2 (NI) 9339**] and
   attending for potential balloon pump vs. urgent CABG
   - appreciate CT [**Doctor First Name 91**] recs
   - continue [**Last Name (LF) **], [**First Name3 (LF) 119**], ACE-i, simvastatin
Label: Direct


Assessment: 51 yr old F with a history of 3V CAD, confirmed on C. cath during this
   admission, 

In [12]:
arr = np.array(prompts) 

In [13]:
idxs = np.random.randint(arr.shape[0], size=5)
for x in idxs:
    print(x)
    print(arr[x])
    print("\n-------------------\n")

4267
Assessment: [**Age over 90 **] yo female with severe dementia with poor functional status, multiple
   medical problems with multiple recent hospitalizations including recent
   ICU stay with intubation, sepsis, pressors, etc, now presents with
   severe dehydration, poor po intake, hypernatremia, and failure to
   thrive.
Plan: ADVANCE DIRECTIVES:
   Health Care Proxy = (niece) [**Name (NI) 2006**] [**Name (NI) 1407**] [**Telephone/Fax (1) 2139**]
   CODE STATUS = FULL CODE
   ICU Care
   Nutrition:
   Glycemic Control:  None
   Lines:
   18 Gauge - [**2106-2-2**] 09:10 AM
   20 Gauge - [**2106-2-2**] 04:39 PM
   Prophylaxis:
   DVT: SQ UF Heparin
   Stress ulcer: Not indicated
   VAP:  Not indicated
Label: Not Relevant

-------------------

278
Assessment: Ms. [**Known lastname **] is a 62 year old female with diabetes, HTN, HL, bipolar
   disorder with multiple prior suicide attempts from medication overdoses
   who is admitted with altered mental status.
Plan: 5.  Bipolar diso

# SciSpacy Dependency Parsing

In [14]:
idxs = np.random.randint(arr.shape[0], size=5)


In [15]:
from scispacy.abbreviation import AbbreviationDetector
Token.set_extension("is_beginning_capitalized", default=False)

In [16]:
# nlp = spacy.load("en_core_sci_scibert")
nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("abbreviation_detector")

In [17]:
doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
	print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

Abbreviation 	 Definition
SBMA 	 (6, 7) Spinal and bulbar muscular atrophy
SBMA 	 (33, 34) Spinal and bulbar muscular atrophy
AR 	 (29, 30) androgen receptor


In [18]:
# docs = ["35 yo F with SLE, restrictive lung disease, CM (EF 15-20%) presents with severe lupus induced cardiomyopathy.",
#         "54 year old man with pmh significant for bipolar disorder presenting with lithium toxicity",
#         "69 F w/ MMP including copd, diastolic chf, dm2, recent prolonged hospitalization where she became trach- dependent secondary to prolonged virulent pseudomonal PNA, presented in septic shock, ARF and AMS.",
#         "73 yo male with history of bilateral renal cell carcinoma metastatic to right adrenalectomy presents for post-operative monitoring after right adrenalectomy and prostate biopsy."]

assessments = [data['Assessment'] for data in dataset['train']]
assessments = np.array(assessments)
assessments[idxs]


array(['79F with h/o DM, HTN, hyperlipidemia, bipolar d/o on lithium,\n   hypothyroidism, presents with symptomatic bradycardia with hypotension\n   s/p temp pacer wire placement.',
       'BACTEREMIA\n ATRIAL FIBRILLATION (AFIB)\n FEVER, UNKNOWN ORIGIN (FUO, HYPERTHERMIA, PYREXIA)',
       'ASSESSMENT AND PLAN: 50 year old M with seizure disorder who presents\n   with episode of syncope.',
       'Mr. [**Known lastname 5979**] is a 57-year-old man with h/o ILD, recent admission for\n   alveolar hemorrhage, also h/o afib, CAD, DM2, HTN who was transferred\n   from [**Location (un) 206**] for hypoxia. He presented to OSH with hypoxia, was\n   intubated.',
       'HYPOXEMIA\n PAIN CONTROL (ACUTE PAIN, CHRONIC PAIN)\n ACTIVITY INTOLERANCE\n ANXIETY\n RESPIRATORY FAILURE, ACUTE (NOT ARDS/[**Doctor Last Name 2**])\n PNEUMONIA, OTHER\n ATRIAL FLUTTER (AFLUTTER)\n   72 year old male with SCC metastatic to left axilla and lung, a fib,\n   HTN, HLD, who presents with shortness of breath and wea

In [19]:
import spacy

doc = nlp("Credit and mortgage account holders must submit their requests")
# span = doc[doc[4].left_edge.i : doc[4].right_edge.i+1]
# with doc.retokenize() as retokenizer:
#     retokenizer.merge(span)
# for token in doc:
#     print(token.text, token.pos_, token.dep_, token.head.text)

/gpfs/ysm/project/rtaylor/vs428/conda_envs/myenv/lib/python3.10/site-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [20]:
processed_docs = []

for idx in idxs:
    doc_string = assessments[idx]
    doc = nlp(str(doc_string))
    left_most_edge = len(doc)
    right_most_edge = 0
    beginning_prob_list = []
    continuing_upper = False
    
    print(f"SENTENCE: {idx}")
    print(doc)
    
    start_token = doc[0]
    if start_token.is_upper:
        continuing_upper = True        
    
    for token_idx, possible_subject in enumerate(doc):

        # extract all the capitalized words at the beginning        
        if continuing_upper and (possible_subject.is_upper or possible_subject.is_punct or possible_subject.is_space):
            beginning_prob_list.append(possible_subject)
            possible_subject._.is_beginning_capitalized = True
            continue
        # print("beginning_prob_list", beginning_prob_list)
        continuing_upper = False
        # print([x for x in possible_subject.head.morph])
        # if possible_subject.dep in [nmod, ccomp] and possible_subject.head.text in ["presents","presenting","presented", "admitted", "transferred"]:
        if possible_subject.dep in [nmod, ccomp, punct, conj] and (possible_subject.head.lemma_ in ["here", "present", "admit", "transfer", "monitor", "transfer", "p/w"]):
            span = doc[possible_subject.left_edge.i : possible_subject.right_edge.i+1]
            if possible_subject.left_edge.i < left_most_edge:
                left_most_edge = possible_subject.left_edge.i
                print(possible_subject.head.lemma_)
            if possible_subject.right_edge.i > right_most_edge:
                right_most_edge = possible_subject.right_edge.i + 1

            # for descendant in possible_subject.subtree:
            #     print(descendant)
            print(span)
            print("--\n")
    
    print("left_most_edge", left_most_edge)
    print("right_most_edge", right_most_edge)    
    print(doc[left_most_edge : right_most_edge])
            
    print("\n---------")
    print("---------")    
    
    processed_docs.append(doc)
    # print("SDFSDFSDF", [token.text for token in doc if not token._.is_beginning_capitalized])
    # processed_docs.append([token.text for token in doc if not token._.is_beginning_capitalized])


SENTENCE: 459
79F with h/o DM, HTN, hyperlipidemia, bipolar d/o on lithium,
   hypothyroidism, presents with symptomatic bradycardia with hypotension
   s/p temp pacer wire placement.
present
,
--

with symptomatic bradycardia with hypotension
--

.
--

left_most_edge 16
right_most_edge 30
, presents with symptomatic bradycardia with hypotension
   s/p temp pacer wire placement.

---------
---------
SENTENCE: 3267
BACTEREMIA
 ATRIAL FIBRILLATION (AFIB)
 FEVER, UNKNOWN ORIGIN (FUO, HYPERTHERMIA, PYREXIA)
left_most_edge 19
right_most_edge 0


---------
---------
SENTENCE: 238
ASSESSMENT AND PLAN: 50 year old M with seizure disorder who presents
   with episode of syncope.
present
with episode of syncope
--

left_most_edge 14
right_most_edge 18
with episode of syncope

---------
---------
SENTENCE: 526
Mr. [**Known lastname 5979**] is a 57-year-old man with h/o ILD, recent admission for
   alveolar hemorrhage, also h/o afib, CAD, DM2, HTN who was transferred
   from [**Location (un) 206**

In [21]:
print(processed_docs[0])
excluding_caps = [token.i for token in processed_docs[0] if (not token._.is_beginning_capitalized) and (not token.is_space)]
# for sent in processed_docs[0].sents:
#     print(sent)

79F with h/o DM, HTN, hyperlipidemia, bipolar d/o on lithium,
   hypothyroidism, presents with symptomatic bradycardia with hypotension
   s/p temp pacer wire placement.


In [22]:
vals = [processed_docs[0][idx].text for idx in range(len(processed_docs[0])) if idx in excluding_caps]

In [24]:
displacy.render(nlp(" ".join(vals)), style="dep", jupyter=True, options={
    # "collapse_phrases":True, 
    "distance":75
})


In [102]:
# for chunk in doc.noun_chunks:
#     print(chunk.text, chunk.root.text, chunk.root.dep_,
#                 chunk.root.head.text)    

In [30]:
for token in processed_docs[1]:
    if token.text == "presenting":
        for descendant in token.subtree:
            print(descendant)
    #     assert subject is descendant or subject.is_ancestor(descendant)
    #     print(descendant.text, descendant.dep_, descendant.n_lefts,
    #             descendant.n_rights,)


In [31]:
for doc_string in docs:
    doc = nlp(doc_string)
    processed_docs.append(doc)
    # print("Abbreviations:", doc._.abbreviations)
    # for abrv in doc._.abbreviations:
    #     print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")    
    for token in doc:
        if token.text in ["presents","presenting","presented", "admitted"]:
            print([subtoken.text for subtoken in token.lefts])  # ['bright', 'red']
            print([subtoken.text for subtoken in token.rights])  # ['on']



NameError: name 'docs' is not defined

In [9]:
bio_clinicalbert_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [10]:
bio_clinicalbert_model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=4)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [11]:
training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch",
                                  
)

In [12]:
import numpy as np
from datasets import load_metric

acc = load_metric("accuracy")
macrof1 = load_metric("f1")

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy":acc.compute(predictions=predictions, references=labels),
            "f1-macro":macrof1.compute(predictions=predictions, references=labels, 
                                       average="macro")}

In [14]:
def tokenize_function(examples):
    return bio_clinicalbert_tokenizer(examples['Assessment'], examples['Plan Subsection'],
                                      truncation="longest_first",
                                      max_length=512,
                                      verbose=True)
                    


In [15]:
dataset = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3942956ec260f7ce.arrow
Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-951b17d1e7bb78b3.arrow


In [16]:
bio_clinicalbert_tokenizer.decode(dataset['valid'][52]['input_ids'])

'[CLS] alteration in nutrition, arousal, attention, and cognition, impaired, balance, impaired, gait, impaired, knowledge, impaired, muscle performace, impaired, transfers, impaired, cholecystitis, acalculous, renal failure, end stage ( end stage renal disease, esrd ), [ * * last name 121 * * ] problem - enter description in comments, altered mental status ( not delirium ) assessment and plan : 78yo m with esrd w / acute cholecystitis s / p perc ccy now w / mdr e. coli infected biliary fluid, now new subacute stroke [SEP] neurologic : poor ms d / t cva on coumadin / afib vs. infection. rect in few days. improving. no narcotics / bzds. b carotid stenosis < 40 %. tte w / o thrombus. [SEP]'

In [17]:
dataset['train'].set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset['valid'].set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [18]:
data_collator = DataCollatorWithPadding(bio_clinicalbert_tokenizer,
                                        max_length=512, 
                                        padding="max_length",
                                        return_tensors="pt" )

In [19]:
trainer = Trainer(
    model=bio_clinicalbert_model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [20]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Assessment, ROW ID, HADM ID, Plan Subsection.
/gpfs/ysm/project/rtaylor/vs428/conda_envs/myenv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4633
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1740


Epoch,Training Loss,Validation Loss,Accuracy,F1-macro
1,0.715300,0.597065,{'accuracy': 0.7487437185929648},{'f1': 0.766732642338792}
2,0.457600,0.672382,{'accuracy': 0.7805695142378559},{'f1': 0.7940059075142675}
3,0.301700,0.918875,{'accuracy': 0.7671691792294807},{'f1': 0.7860078751767712}


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Assessment, ROW ID, HADM ID, Plan Subsection.
***** Running Evaluation *****
  Num examples = 597
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Assessment, ROW ID, HADM ID, Plan Subsection.
***** Running Evaluation *****
  Num examples = 597
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpo

TrainOutput(global_step=1740, training_loss=0.45485844447694973, metrics={'train_runtime': 821.007, 'train_samples_per_second': 16.929, 'train_steps_per_second': 2.119, 'total_flos': 3657046227554304.0, 'train_loss': 0.45485844447694973, 'epoch': 3.0})